# Capítulo 06 - Integração com Python

Este notebook explora a integração profunda entre DuckDB e Python, mostrando como trabalhar com DataFrames, conexões, configurações e práticas recomendadas.

## 📚 Tópicos Abordados:
1. Queries Básicas e Relations
2. Leitura de Arquivos (CSV, Parquet, JSON)
3. Integração com Pandas, Polars e Arrow
4. Conversões de Resultados
5. Exportação de Dados
6. Gestão de Conexões
7. Configurações Avançadas
8. Casos de Uso Práticos (ETL, Análise)
9. Boas Práticas
10. Extensões e Performance

## 1. Setup e Preparação

In [ ]:
# Instalar dependências
%pip install duckdb pandas polars pyarrow -q
print("✓ Pacotes instalados!")

In [ ]:
import duckdb
import pandas as pd
import json
import os
from datetime import datetime, timedelta

print(f"DuckDB versão: {duckdb.__version__}")
print(f"Pandas versão: {pd.__version__}")
print("\n✓ Imports realizados!")

## 2. Queries Básicas

### 2.1 Query Simples

In [ ]:
# Query simples retorna um Relation
result = duckdb.sql("SELECT 42 AS answer, 'DuckDB' AS name")

print("Tipo do resultado:", type(result))
print("\nResultado:")
result.show()

print("\n✓ Query executada!")

### 2.2 Encadeamento de Queries

DuckDB permite referenciar resultados anteriores!

In [ ]:
# Primeira query
r1 = duckdb.sql("SELECT 42 AS i, 'Python' AS lang")

# Segunda query referencia r1!
print("Dobrando o valor de i:")
duckdb.sql("SELECT i * 2 AS doubled, lang FROM r1").show()

# Terceira query
print("\nTriplicando o valor:")
duckdb.sql("SELECT i * 3 AS tripled FROM r1").show()

print("\n✓ Relations podem ser reutilizadas!")

## 3. Leitura de Arquivos

### 3.1 Criar Arquivos de Teste

In [ ]:
# Criar CSV
csv_data = """id,nome,idade,cidade
1,Alice,28,São Paulo
2,Bob,35,Rio de Janeiro
3,Charlie,42,Belo Horizonte"""

with open('example.csv', 'w', encoding='utf-8') as f:
    f.write(csv_data)

# Criar Parquet
duckdb.sql("""
    COPY (SELECT * FROM 'example.csv') 
    TO 'example.parquet'
""")

# Criar JSON
json_data = [
    {"id": 1, "nome": "Alice", "idade": 28, "cidade": "São Paulo"},
    {"id": 2, "nome": "Bob", "idade": 35, "cidade": "Rio de Janeiro"},
    {"id": 3, "nome": "Charlie", "idade": 42, "cidade": "Belo Horizonte"}
]

with open('example.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f)

print("✓ Arquivos de teste criados: CSV, Parquet, JSON")

### 3.2 Ler CSV

In [ ]:
# Método 1: read_csv()
print("Método 1 - read_csv():")
rel = duckdb.read_csv("example.csv")
print(f"Tipo: {type(rel)}")
rel.show()

# Método 2: SQL direto
print("\nMétodo 2 - SQL direto:")
duckdb.sql("SELECT * FROM 'example.csv' WHERE idade > 30").show()

print("\n✓ CSV lido de duas formas!")

### 3.3 Ler Parquet

In [ ]:
# read_parquet()
print("Lendo Parquet:")
rel = duckdb.read_parquet("example.parquet")
rel.show()

# SQL direto
print("\nQuery SQL:")
duckdb.sql("""
    SELECT nome, cidade 
    FROM 'example.parquet' 
    ORDER BY idade DESC
""").show()

### 3.4 Ler JSON

In [ ]:
# read_json()
print("Lendo JSON:")
rel = duckdb.read_json("example.json")
rel.show()

# SQL direto com agregação
print("\nAgregação:")
duckdb.sql("""
    SELECT 
        COUNT(*) AS total,
        AVG(idade) AS idade_media
    FROM 'example.json'
""").show()

## 4. Integração com Frameworks Python

### 4.1 Pandas DataFrame

In [ ]:
# Criar conexão para uso nas próximas células
con = duckdb.connect()

# Criar DataFrame Pandas com tipos explícitos
pandas_df = pd.DataFrame({
    "produto": pd.Series(["Notebook", "Mouse", "Teclado"], dtype="object"),
    "preco": [3500.00, 50.00, 150.00],
    "estoque": [15, 120, 80]
})

print("DataFrame Pandas:")
print(pandas_df)

# Consulta SQL no DataFrame
print("\nConsulta SQL no DataFrame:")
result = con.execute("""
    SELECT 
        produto,
        preco,
        preco * estoque AS valor_total_estoque
    FROM pandas_df
    ORDER BY valor_total_estoque DESC
""").fetchdf()

print(result)
print("\n✓ DuckDB consulta Pandas diretamente!")

### 4.2 Polars DataFrame

In [ ]:
import polars as pl

# Criar DataFrame Polars
polars_df = pl.DataFrame({
    "categoria": ["Eletrônicos", "Móveis", "Livros"],
    "vendas": [125000, 85000, 42000]
})

print("DataFrame Polars:")
print(polars_df)

# Consultar Polars DataFrame
print("\nConsulta SQL:")
duckdb.sql("""
    SELECT 
        categoria,
        vendas,
        vendas * 1.15 AS com_impostos
    FROM polars_df
""").show()

print("\n✓ DuckDB consulta Polars também!")

### 4.3 PyArrow Table

In [ ]:
import pyarrow as pa

# Criar Arrow Table
arrow_table = pa.Table.from_pydict({
    "id": [1, 2, 3],
    "nome": ["Alice", "Bob", "Charlie"],
    "score": [95.5, 87.0, 91.5]
})

print("Arrow Table:")
print(arrow_table)

# Consultar Arrow Table
print("\nConsulta SQL:")
duckdb.sql("""
    SELECT nome, score,
        CASE 
            WHEN score >= 90 THEN 'A'
            WHEN score >= 80 THEN 'B'
            ELSE 'C'
        END AS conceito
    FROM arrow_table
""").show()

print("\n✓ Arrow Tables suportadas!")

## 5. Conversões de Resultados

### 5.1 Para Lista de Tuplas

In [ ]:
result = duckdb.sql("""
    SELECT 42 AS num, 'hello' AS text, 3.14 AS pi
""")

# fetchall() - todas as linhas
rows = result.fetchall()
print("fetchall():")
print(rows)
print(f"Tipo: {type(rows[0])}")

# fetchone() - uma linha
result2 = duckdb.sql("SELECT 1, 2, 3")
row = result2.fetchone()
print("\nfetchone():")
print(row)

# fetchmany() - N linhas
result3 = duckdb.sql("SELECT * FROM range(10)")
some_rows = result3.fetchmany(3)
print("\nfetchmany(3):")
print(some_rows)

### 5.2 Para Pandas DataFrame

In [ ]:
result = duckdb.sql("""
    SELECT * FROM 'example.csv'
""")

# Converter para Pandas
df = result.df()

print(f"Tipo: {type(df)}")
print(f"\nShape: {df.shape}")
print("\nDataFrame:")
print(df)

# Usar métodos Pandas
print("\nEstatísticas:")
print(df['idade'].describe())

### 5.3 Para Polars DataFrame

In [ ]:
result = duckdb.sql("""
    SELECT * FROM 'example.csv'
""")

# Converter para Polars
pl_df = result.pl()

print(f"Tipo: {type(pl_df)}")
print(f"\nShape: {pl_df.shape}")
print("\nDataFrame Polars:")
print(pl_df)

print("\n✓ Convertido para Polars!")

### 5.4 Para Arrow Table

In [ ]:
result = duckdb.sql("""
    SELECT * FROM 'example.csv'
""")

# Converter para Arrow
arrow = result.arrow()

print(f"Tipo: {type(arrow)}")
print(f"\nSchema: {arrow.schema}")

# Ler os batches
arrow_table = arrow.read_all()
print(f"\nNum rows: {len(arrow_table)}")
print("\nArrow Table:")
print(arrow_table)

### 5.5 Para NumPy Arrays

In [ ]:
result = duckdb.sql("""
    SELECT 42 AS num, 100 AS total, 3.14 AS pi
""")

# Converter para NumPy (dict de arrays)
numpy_result = result.fetchnumpy()

print("fetchnumpy():")
print(numpy_result)
print(f"\nTipo de 'num': {type(numpy_result['num'])}")
print(f"Valores: {numpy_result['num'][0]}, {numpy_result['total'][0]}")

## 6. Exportação de Dados

### 6.1 Exportar como Parquet

In [ ]:
# Método 1: COPY
duckdb.sql("""
    COPY (
        SELECT * FROM 'example.csv' WHERE idade > 30
    ) TO 'output1.parquet'
""")

print("✓ Método 1 (COPY): output1.parquet criado")
print(f"Tamanho: {os.path.getsize('output1.parquet'):,} bytes")

# Verificar
print("\nConteúdo:")
duckdb.sql("SELECT * FROM 'output1.parquet'").show()

### 6.2 Exportar como CSV

In [ ]:
# Exportar CSV
duckdb.sql("""
    COPY (
        SELECT nome, idade FROM 'example.csv'
    ) TO 'output.csv' (HEADER)
""")

print("✓ output.csv criado")
print(f"Tamanho: {os.path.getsize('output.csv'):,} bytes")

# Mostrar conteúdo
with open('output.csv', 'r', encoding='utf-8') as f:
    print("\nConteúdo:")
    print(f.read())

### 6.3 Exportar como JSON

In [ ]:
# Exportar JSON
duckdb.sql("""
    COPY (
        SELECT * FROM 'example.csv'
    ) TO 'output.json' (FORMAT JSON, ARRAY true)
""")

print("✓ output.json criado")
print(f"Tamanho: {os.path.getsize('output.json'):,} bytes")

# Verificar
with open('output.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(f"\nRegistros: {len(data)}")
    print(json.dumps(data[0], indent=2, ensure_ascii=False))

## 7. Gestão de Conexões

### 7.1 Conexão em Memória

In [ ]:
# Criar conexão em memória
con = duckdb.connect()

print("Conexão criada!")
con.sql("SELECT 42 AS answer").show()

# Fechar conexão
con.close()
print("\n✓ Conexão fechada")

### 7.2 Conexão com Arquivo

In [ ]:
# Criar/conectar a arquivo de banco de dados
con = duckdb.connect("test.db")

# Criar tabela
con.sql("DROP TABLE IF EXISTS usuarios")
con.sql("CREATE TABLE usuarios (id INTEGER, nome VARCHAR)")
con.sql("INSERT INTO usuarios VALUES (1, 'Alice'), (2, 'Bob')")

print("Dados persistidos em test.db:")
con.table("usuarios").show()

con.close()
print("\n✓ Dados salvos em disco!")

### 7.3 Context Manager (Recomendado)

In [ ]:
# Usar 'with' para fechar automaticamente
with duckdb.connect("test2.db") as con:
    con.sql("DROP TABLE IF EXISTS produtos")
    con.sql("CREATE TABLE produtos (id INTEGER, nome VARCHAR, preco DOUBLE)")
    con.sql("INSERT INTO produtos VALUES (1, 'Notebook', 3500.00)")
    
    print("Dentro do context manager:")
    con.table("produtos").show()

# Conexão fechada automaticamente!
print("\n✓ Conexão fechada automaticamente pelo 'with'")

### 7.4 Configurações na Conexão

In [ ]:
# Configurar número de threads
con = duckdb.connect(config={'threads': 4})
print("✓ Conexão com 4 threads")
con.close()

# Múltiplas configurações
con = duckdb.connect(config={
    'threads': 2,
    'max_memory': '1GB',
    'default_order': 'DESC'
})

print("\nConfigurações aplicadas:")
print("- Threads: 2")
print("- Memória máxima: 1GB")
print("- Ordem padrão: DESC")

# Testar ordem padrão
con.sql("SELECT * FROM range(5)").show()
con.close()

## 8. Casos de Uso Práticos

### 8.1 Análise de Vendas com Pandas

In [ ]:
# Criar dados de vendas
vendas = pd.DataFrame({
    'data': pd.date_range('2024-01-01', periods=100),
    'produto': pd.Series(['A', 'B', 'C'] * 33 + ['A'], dtype="object"),
    'quantidade': range(1, 101),
    'preco': [10.5, 20.0, 15.75] * 33 + [10.5]
})

print(f"Dataset: {len(vendas)} vendas")
print("\nPrimeiras linhas:")
print(vendas.head())

# Análise com DuckDB usando conexão
resultado = con.execute("""
    SELECT
        produto,
        COUNT(*) as total_vendas,
        SUM(quantidade) as total_quantidade,
        ROUND(SUM(quantidade * preco), 2) as receita_total,
        ROUND(AVG(preco), 2) as preco_medio
    FROM vendas
    GROUP BY produto
    ORDER BY receita_total DESC
""").fetchdf()

print("\n📊 Análise por Produto:")
print(resultado)

### 8.2 Pipeline ETL Completo

In [ ]:
# Criar dados de exemplo
# Vendas CSV
vendas_data = pd.DataFrame({
    'venda_id': range(1, 11),
    'cliente_id': [1, 2, 1, 3, 2, 1, 3, 2, 1, 3],
    'data': pd.date_range('2024-01-01', periods=10),
    'valor_total': [100, 250, 150, 300, 200, 180, 220, 190, 160, 280],
    'quantidade': [2, 5, 3, 6, 4, 3, 5, 4, 3, 6]
})
vendas_data.to_csv('vendas.csv', index=False)

# Clientes Parquet
clientes_data = pd.DataFrame({
    'id': [1, 2, 3],
    'cliente_nome': pd.Series(['Alice', 'Bob', 'Charlie'], dtype="object"),
    'regiao': pd.Series(['Sul', 'Sudeste', 'Norte'], dtype="object")
})
con.execute("COPY clientes_data TO 'clientes.parquet'")

print("✓ Dados de exemplo criados\n")

# Pipeline ETL
with duckdb.connect() as con_etl:
    print("🔄 Iniciando ETL...")
    
    # Extract: Ler de múltiplas fontes
    con_etl.sql("CREATE TABLE vendas AS SELECT * FROM 'vendas.csv'")
    con_etl.sql("CREATE TABLE clientes AS SELECT * FROM 'clientes.parquet'")
    print("  ✓ Extract: Dados carregados")
    
    # Transform: Processar e limpar dados
    con_etl.sql("""
        CREATE TABLE vendas_processadas AS
        SELECT
            v.venda_id,
            v.data,
            c.cliente_nome,
            c.regiao,
            v.valor_total,
            v.quantidade,
            v.valor_total / v.quantidade AS preco_unitario
        FROM vendas v
        JOIN clientes c ON v.cliente_id = c.id
        WHERE v.data >= '2024-01-01'
    """)
    print("  ✓ Transform: Dados processados")
    
    # Load: Exportar para Parquet
    con_etl.sql("""
        COPY vendas_processadas
        TO 'vendas_processadas.parquet'
        (FORMAT parquet, COMPRESSION zstd)
    """)
    print("  ✓ Load: Exportado para vendas_processadas.parquet")
    
    # Visualizar resultado
    print("\n📊 Resultado ETL:")
    con_etl.sql("SELECT * FROM vendas_processadas LIMIT 5").show()
    
    print(f"\n✓ ETL concluído! Arquivo: {os.path.getsize('vendas_processadas.parquet'):,} bytes")

### 8.3 Análise Rápida em Notebooks

In [ ]:
# Criar dados de exemplo
sales_df = pd.DataFrame({
    'category': pd.Series(['Electronics', 'Electronics', 'Furniture', 'Books', 'Electronics', 'Furniture'], dtype="object"),
    'product': pd.Series(['Laptop', 'Mouse', 'Chair', 'Novel', 'Keyboard', 'Desk'], dtype="object"),
    'price': [1200, 25, 350, 15, 75, 600],
    'quantity': [5, 50, 20, 100, 30, 10],
    'revenue': [6000, 1250, 7000, 1500, 2250, 6000]
})

print("Dataset de vendas:")
print(sales_df)

# Análise rápida com DuckDB usando conexão
print("\n📊 Análise por Categoria:")
con.execute("""
    SELECT
        category,
        COUNT(*) as count,
        ROUND(AVG(price), 2) as avg_price,
        SUM(revenue) as total_revenue
    FROM sales_df
    GROUP BY category
    ORDER BY total_revenue DESC
""").fetchdf().head()

print("\n✓ Análise instantânea!")

### 8.4 Combinar Dados em Memória e Disco

In [ ]:
# Dados em memória (Pandas)
clientes_memoria = pd.DataFrame({
    'id': [1, 2, 3],
    'nome': pd.Series(['Alice Silva', 'Bob Santos', 'Charlie Oliveira'], dtype="object")
})

print("Clientes em memória:")
print(clientes_memoria)

# Combinar com arquivo no disco usando conexão
print("\n🔗 Combinando dados (memória + disco):")
resultado = con.execute("""
    SELECT
        v.venda_id,
        c.nome,
        v.valor_total,
        v.data
    FROM 'vendas.csv' v
    JOIN clientes_memoria c ON v.cliente_id = c.id
    WHERE v.valor_total > 150
    ORDER BY v.data DESC
    LIMIT 5
""").fetchdf()

print(resultado)
print("\n✓ JOIN entre memória e disco!")

## 9. Boas Práticas

### 9.1 Reutilizar Conexões

In [ ]:
import time

# ✅ BOM: Reutilizar conexão
start = time.time()
con = duckdb.connect()
for i in range(100):
    con.sql(f"SELECT {i}").fetchall()
con.close()
time_good = time.time() - start

print(f"✅ Reutilizando conexão: {time_good*1000:.2f}ms")

# ❌ RUIM: Criar nova conexão toda vez
start = time.time()
for i in range(100):
    con = duckdb.connect()
    con.sql(f"SELECT {i}").fetchall()
    con.close()
time_bad = time.time() - start

print(f"❌ Criando nova conexão: {time_bad*1000:.2f}ms")
print(f"\n💡 Reutilizar é {time_bad/time_good:.1f}x mais rápido!")

### 9.2 Usar Parquet ao Invés de CSV

In [ ]:
# Criar dataset teste
with duckdb.connect() as con:
    con.sql("""
        COPY (
            SELECT 
                i AS id,
                'User_' || i AS nome,
                20 + (i % 50) AS idade
            FROM range(1, 10001) t(i)
        ) TO 'benchmark.csv'
    """)
    
    con.sql("""
        COPY (
            SELECT * FROM 'benchmark.csv'
        ) TO 'benchmark.parquet'
    """)

csv_size = os.path.getsize('benchmark.csv')
parquet_size = os.path.getsize('benchmark.parquet')

print("Tamanhos:")
print(f"CSV:     {csv_size:>10,} bytes")
print(f"Parquet: {parquet_size:>10,} bytes ({parquet_size/csv_size*100:.1f}%)")

# Benchmark leitura
print("\n⚡ Benchmark Leitura:")

# CSV
start = time.time()
duckdb.sql("SELECT COUNT(*) FROM 'benchmark.csv'").fetchall()
time_csv = time.time() - start
print(f"CSV:     {time_csv*1000:>8.2f}ms")

# Parquet
start = time.time()
duckdb.sql("SELECT COUNT(*) FROM 'benchmark.parquet'").fetchall()
time_parquet = time.time() - start
print(f"Parquet: {time_parquet*1000:>8.2f}ms")

print(f"\n💡 Parquet é {time_csv/time_parquet:.1f}x mais rápido!")

### 9.3 Context Managers e Limpeza

In [ ]:
# ✅ BOM: Usar 'with' para garantir fechamento
def processar_dados_correto():
    with duckdb.connect('temp.db') as con:
        con.sql("CREATE TABLE test (id INTEGER)")
        con.sql("INSERT INTO test VALUES (1), (2), (3)")
        result = con.table('test').fetchall()
    # Conexão fechada automaticamente
    return result

result = processar_dados_correto()
print("✅ Context manager:")
print(result)

# ❌ RUIM: Esquecer de fechar pode causar problemas
def processar_dados_errado():
    con = duckdb.connect('temp2.db')
    con.sql("CREATE TABLE test (id INTEGER)")
    result = con.table('test').fetchall()
    # Sem con.close() - pode causar problemas!
    return result

print("\n❌ Sem fechar: Pode causar problemas")
print("💡 Sempre use 'with' ou feche explicitamente!")

## 10. Recursos Avançados

### 10.1 Prepared Statements

In [ ]:
con_prep = duckdb.connect()

# DuckDB 1.4.3 não tem prepare(), usar execute com parâmetros
print("Queries parametrizadas:")

# Exemplo 1
result1 = con_prep.execute("SELECT ? * ? AS resultado", [2, 3]).fetchall()
print(f"2 * 3 = {result1[0][0]}")

# Exemplo 2
result2 = con_prep.execute("SELECT ? * ? AS resultado", [5, 7]).fetchall()
print(f"5 * 7 = {result2[0][0]}")

# Exemplo 3
result3 = con_prep.execute("SELECT ? * ? AS resultado", [10, 10]).fetchall()
print(f"10 * 10 = {result3[0][0]}")

con_prep.close()
print("\n✓ Queries parametrizadas executadas!")

### 10.2 Limitar Memória

In [ ]:
# Processar com limite de memória
with duckdb.connect() as con:
    # Configurar memória máxima
    con.execute("SET memory_limit='500MB'")
    
    print("✓ Memória limitada a 500MB")
    
    # Query que respeita o limite
    result = con.sql("""
        SELECT
            (i % 12) + 1 AS mes,
            COUNT(*) as eventos,
            AVG(i) as media
        FROM range(1, 100001) t(i)
        GROUP BY mes
        ORDER BY mes
    """).df()
    
    print("\nResultado:")
    print(result.head())
    print("\n💡 DuckDB gerencia memória automaticamente!")

## 🎯 Resumo do Capítulo

Neste capítulo exploramos:

### ✅ Funcionalidades:
1. **Queries SQL**: Básicas e encadeadas
2. **Leitura**: CSV, Parquet, JSON
3. **Integração**: Pandas, Polars, PyArrow
4. **Conversões**: tuplas, DataFrame, Arrow, NumPy
5. **Exportação**: Parquet, CSV, JSON
6. **Conexões**: Memória, arquivo, context manager
7. **ETL**: Pipeline completo
8. **Performance**: Reutilizar conexões, usar Parquet

### 🔑 Pontos-Chave:
- DuckDB consulta **DataFrames diretamente**
- Relations podem ser **reutilizadas**
- **Context manager** (`with`) é recomendado
- **Parquet** é mais rápido que CSV
- Reutilizar conexões **melhora performance**
- Suporta **Pandas, Polars e Arrow**

### 💡 Boas Práticas:
1. Use `with` para conexões
2. Reutilize conexões
3. Prefira Parquet a CSV
4. Limite memória quando necessário
5. Use prepared statements para queries repetidas

### 📚 Próximo Capítulo:
Tipos de dados no DuckDB!

## 🧹 Limpeza (Opcional)

In [ ]:
# Remover arquivos criados
arquivos = [
    'example.csv', 'example.parquet', 'example.json',
    'output1.parquet', 'output.csv', 'output.json',
    'test.db', 'test2.db', 'temp.db', 'temp2.db',
    'vendas.csv', 'clientes.parquet', 'vendas_processadas.parquet',
    'benchmark.csv', 'benchmark.parquet', 'temp_python_data.json'
]

for arquivo in arquivos:
    if os.path.exists(arquivo):
        os.remove(arquivo)
        print(f"✓ Removido: {arquivo}")

print("\n✓ Limpeza concluída!")